In [2]:
import os
import json
import torch
import warnings
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

# 경고 메시지 숨기기 (깔끔한 출력을 위해)
warnings.filterwarnings("ignore")

# MPS (Apple Silicon) 장치 확인
if torch.backends.mps.is_available():
    device = "mps"
    print(f"✅ Apple Silicon (M3 Pro) 감지됨: MPS 가속을 사용합니다.")
else:
    device = "cpu"
    print("⚠️ MPS 미감지. CPU로 실행됩니다 (느릴 수 있음).")

✅ Apple Silicon (M3 Pro) 감지됨: MPS 가속을 사용합니다.


In [ ]:
import json
from datasets import Dataset

# 1. 파일 읽기
file_path = "train_data.json"
with open(file_path, "r", encoding="utf-8") as f:
    # 줄바꿈 된 JSONL 형식이든, 리스트 형식이든 다 처리하도록 로직 보완
    try:
        raw_data = [json.loads(line) for line in f]
    except json.JSONDecodeError:
        f.seek(0)
        raw_data = json.load(f)

# 2. 프롬프트 포맷팅 (모든 키 활용)
system_prompt = "너는 영미권의 최신 슬랭과 밈(Meme)을 한국의 인터넷 문화와 정서에 맞게 '초월 번역'하는 전문 번역가야."

formatted_data = []
for item in raw_data:
    # (1) Idiom 정보 추출
    term = item['idiom']['term']
    meaning = item['idiom']['meaning']
    
    # (2) 풍부한 User Input 구성
    # 지시사항 + 관용구 힌트 + 실제 문장을 하나로 합칩니다.
    # 이렇게 하면 모델이 "아, 이 단어는 이런 뜻이니까 이렇게 번역해야지"라고 추론하는 과정을 돕습니다.
    full_user_input = (
        f"지시사항: {item['instruction']}\n\n"
        f"[참고 표현]\n"
        f"- 표현: {term}\n"
        f"- 의미: {meaning}\n\n"
        f"입력 문장: {item['input']}"
    )

    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": full_user_input},
        {"role": "assistant", "content": item['output']}
    ]
    formatted_data.append({"messages": conversation})

# 3. 데이터셋 생성 및 분할
dataset = Dataset.from_list(formatted_data)
dataset = dataset.train_test_split(test_size=0.1)

print(f"📊 데이터 준비 완료")
print(f"   - 학습 데이터: {len(dataset['train'])}개")
print(f"   - 검증 데이터: {len(dataset['test'])}개")
print("="*50)
print(f"[구성된 프롬프트 예시]:\n{dataset['train'][0]['messages'][1]['content']}")
print("="*50)

📊 데이터 준비 완료
   - 학습 데이터: 540개
   - 검증 데이터: 60개
[구성된 프롬프트 예시]:
지시사항: Don't translate it in Korean, but translate it according to Korean culture

[참고 표현]
- 표현: Cut corners
- 의미: 대충 하다, 절차를 무시하다

입력 문장: Cutting corners led to the accident.


In [4]:
model_id = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드 (Float32 for Stable MPS Training)
print("📥 모델 로딩 중... (인터넷 속도에 따라 시간이 걸릴 수 있습니다)")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32, 
    trust_remote_code=True
)
# 주의: Trainer가 자동으로 device를 관리하므로 수동으로 .to(device) 하지 않음

print("✅ 모델 및 토크나이저 로드 완료!")

📥 모델 로딩 중... (인터넷 속도에 따라 시간이 걸릴 수 있습니다)


`torch_dtype` is deprecated! Use `dtype` instead!


✅ 모델 및 토크나이저 로드 완료!


In [5]:
peft_config = LoraConfig(
    r=16,            # 랭크 (표현력)
    lora_alpha=32,   # 스케일링 계수
    lora_dropout=0.05,
    target_modules="all-linear", # 모든 선형 레이어에 적용 (가장 효과 좋음)
    bias="none",
    task_type="CAUSAL_LM",
)
print("🔧 LoRA 설정 완료")

🔧 LoRA 설정 완료


In [8]:
output_dir = "./hyperclova-translator-mps"

training_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=5,             # 데이터가 적으므로 5~10회 반복 권장
    per_device_train_batch_size=4,  # 배치 사이즈
    gradient_accumulation_steps=2,  # 그래디언트 누적
    learning_rate=2e-2,             # 학습률
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=1,                # 매 스텝마다 Loss 출력 (과제용 그래프 그리기에 좋음)
    eval_strategy="steps",
    eval_steps=10,                  # 10 스텝마다 검증
    save_strategy="steps",
    save_steps=50,
    
    # [핵심] M3 Pro 호환성을 위해 Mixed Precision 끄기
    bf16=False,
    fp16=False,
    
    use_cpu=False,
    report_to="none",
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
    args=training_args,
    peft_config=peft_config,
)

print("🚀 학습 시작! (Loss가 떨어지는지 확인하세요)")
trainer.train()

# 학습된 어댑터 저장
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"🎉 학습 완료! 모델이 {output_dir}에 저장되었습니다.")

Truncating eval dataset: 100%|██████████| 60/60 [00:00<00:00, 39346.19 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 100275}.


🚀 학습 시작! (Loss가 떨어지는지 확인하세요)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
10,1.329400,1.842246,1.661859,10224.000000,0.686159
20,12.707800,9.564654,4.758411,20458.000000,0.031746
30,8.303900,7.762658,7.511355,30687.000000,0.016618
40,6.783900,6.618316,6.570342,41022.000000,0.055310
50,6.250100,6.340600,6.215347,51216.000000,0.055310
60,6.416200,6.244977,6.320243,61426.000000,0.055310
70,6.343000,6.196061,6.255406,71118.000000,0.055310
80,5.943700,6.165631,6.231468,81212.000000,0.055310
90,6.420700,6.147174,6.216238,91633.000000,0.055310
100,6.071800,6.133157,6.287160,101783.000000,0.055310


KeyboardInterrupt: 

In [7]:
from peft import PeftModel # 라이브러리 확실하게 다시 임포트

# 테스트할 문장 리스트
test_sentences = [
    "She’s not just late, she runs on her own Disney time.",
    "They expected him to crack under pressure, but he stayed cool as a cucumber.",
    "He’s the golden boy of the company, so management treats him like he can do no wrong.",
    "Trying to get them to finish the report is like herding cats.",
    "When she apologized, it sounded like one of those PR-approved celebrity apology notes.",
    "She’s got resting politician face, always looks like she’s about to debate someone.",
    "He’s reading way too much into it-classic tinfoil-hat energy.",
    "They acted like the Fourth of July over the tiniest bit of praise.",
    "He’s basically the office Santa Claus—shows up rarely but always hands out gifts."
]

print(f"{'='*20} [결과 비교 분석] {'='*20}\n")

output_dir = "./hyperclova-translator-mps"

# 저장된 모델과 토크나이저 불러오기
print("📥 저장된 모델 로딩 중...")
model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float32,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ 모델 및 토크나이저 로드 완료!")

# [핵심 수정] 저장된 LoRA 어댑터를 Base Model에 확실하게 장착합니다!
# 이렇게 해야 model이 'LlamaForCausalLM'에서 'PeftModel'로 변신합니다.
print("🔧 LoRA 어댑터 로드 중...")
model = PeftModel.from_pretrained(model, output_dir)

# 추론을 위해 평가 모드 전환 및 GPU 이동
model.eval()
model.to(device)

for i, sentence in enumerate(test_sentences):
    # 1. 입력 프롬프트 구성
    full_input = instruction_prefix + sentence
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": full_input}
    ]
    formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    print(f"📝 Test Case {i+1}: {sentence}")

    # --- Case 1: 학습 전 (LoRA 끄기) ---
    # 이제 model은 PeftModel이므로 disable_adapter()가 작동합니다.
    with model.disable_adapter():
        with torch.no_grad():
            outputs_base = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.5,
                pad_token_id=tokenizer.eos_token_id
            )
        # 결과 파싱
        decoded_base = tokenizer.decode(outputs_base[0], skip_special_tokens=True)
        try:
            base_result = decoded_base.split("assistant")[-1].strip()
        except:
            base_result = decoded_base
        print(f"   ❌ [Before]: {base_result}")

    # --- Case 2: 학습 후 (LoRA 켜기) ---
    # context manager를 빠져나오면 자동으로 다시 활성화됩니다.
    with torch.no_grad():
        outputs_tuned = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    # 결과 파싱
    decoded_tuned = tokenizer.decode(outputs_tuned[0], skip_special_tokens=True)
    try:
        tuned_result = decoded_tuned.split("assistant")[-1].strip()
    except:
        tuned_result = decoded_tuned
    print(f"   ✅ [After ]: {tuned_result}\n")
    
    print("-" * 60)

==================== [결과 비교 분석] ====================

📥 저장된 모델 로딩 중...
✅ 모델 및 토크나이저 로드 완료!
🔧 LoRA 어댑터 로드 중...


NameError: name 'instruction_prefix' is not defined